<a href="https://colab.research.google.com/github/cinqbox/liquipedia-esports/blob/main/liquipedia%E3%80%80%E3%83%87%E3%83%BC%E3%82%BF%E5%8F%8E%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import requests
import json
import pprint
import time
import re
import pandas as pd
import numpy as np


In [20]:
# 本文中から特定の部分だけを抽出
def Re(pattern, body):
    ans = "".join(re.findall(pattern, body, flags=re.MULTILINE))
    return ans

# 大会ごとのデータ取得
def getByCompInfo(title_list, url):
    result_df = pd.DataFrame()

    for title in title_list:
        params = {"action": "query",
                "format": "json",
                "prop": "revisions",
                "rvprop": "content|ids",
                "titles": title}

        S = requests.Session()
        R = S.get(url=url, params=params)
        DATA = R.json()
        ids = list(DATA["query"]["pages"].keys())[0]
        body = DATA["query"]["pages"][ids]["revisions"][0]['*']

        pattern_prize = r"^\|prizepool=(.*?)$"
        pattern_type = r"^\|type=(.*?)$"
        pattern_country = r"^\|country=(.*?)$"
        pattern_prize_local = r"^\|localcurrency=(.*?)$"
        pattern_team_number = r"^\|team_number=(.*?)$"
        pattern_name = r"^\|name=(.*?)$"
        pattern_prize_usd = r"^\|prizepoolusd=(.*?)$"

        prize_local = Re(pattern_prize_local, body)
        holdtype = Re(pattern_type, body)
        team_number = Re(pattern_team_number, body)
        country = Re(pattern_country, body)
        name = Re(pattern_name, body)

        # 英語
        if prize_local == "":
          prize = Re(pattern_prize_usd, body)
          prize_local = "usd"
        else:
          prize = Re(pattern_prize, body)

        dic = {"大会名": name, "賞金": prize, "通貨": prize_local, "開催方法": holdtype, "チーム数": team_number, "開催地域": country}
        
        df = pd.DataFrame.from_dict(dic, orient='index').T
        result_df = pd.concat([result_df, df], ignore_index=True)
        time.sleep(15)
    return result_df


In [21]:
# データの整形、csv形式にエクスポート
def Preprocessing(game_title, result_df):
    result_df["通貨"] = result_df["通貨"].str.upper()
    result_df = result_df[~result_df["大会名"].duplicated()]
    result_df.to_csv(game_title + ".csv", index=False)


In [22]:
def getGameInfo(game_title):
    S = requests.Session()
    url = "https://liquipedia.net/" + game_title + "/api.php"
    params = {"action": "query",
              "format": "json",
              "list": "categorymembers",
              "cmprop": "title",
              "cmlimit": "100",
              "cmtitle": "Category:Japanese Tournaments"}
    R = S.get(url=url, params=params)
    DATA = R.json()
    time.sleep(10)

    # 日本大会一覧を抽出
    title_list = []
    title = DATA["query"]["categorymembers"]
    for i in title:
      title_list.append(i["title"])

    result_df = getByCompInfo(title_list, url)
    Preprocessing(game_title, result_df)
    print("CSV SAVED!!!")


    

In [23]:
getGameInfo("rocketleague")

User:AdliWave/Gulliver Aegis Informal Tournament
User:AndiiRL/Intel World Open Fantasy
DeToNator Invitational/1
EGG eSports Challenge/Gulliver Cup/2020
EGG eSports Challenge/Gulliver Cup/2020/Qualifier 1
EGG eSports Challenge/Gulliver Cup/2020/Qualifier 2
User:Hawnty/Logicool G Cup 2018/Finals
User:Hawnty/Logicool G Cup 2018/Overview
User:Hawnty/Logicool G Cup 2018/Qualifiers
User:Hawnty/LogicoolGCup2018
Intel World Open
Intel World Open/Japan
Logicool/G Cup/2018
Logicool/G Cup/2018/Qualifier
User:NishChahar/primal
User:NishChahar/PrimalQuals
Primal/Rocket League Japan Series
Primal/Rocket League Japan Series/League Play
Primal/Rocket League Japan Series/Qualifier 1
Primal/Rocket League Japan Series/Qualifier 2
CSV SAVED!!!
